# Table of Contents (Explore Azure Machine Learning SDK)
____
**1. [Workspace](#Workspace)**

- 1.1 Create a Workspace and write config.json
- 1.2 Connect to Workspace with config.json
- 1.3 Connect to Workspace with variable
- 1.4 Get details of the Workspace

**2. [Experiment](#Experiment)**

- 2.1 Create/Get Experiment

**3.  [Run](#Run) **

- 3.1 Create a run object in the experiment and start logging
- 3.2 Scalar

**4. [Model](#Model)**

- 4.1 Register Model
- 4.2 Download Model
- 4.3 Delete Model

**5. [ComputeTarget, RunConfiguration, and ScriptRunConfig](#ComputeTarget,RunConfiguration,andScriptRunConfig)**

- 5.1 Setting up an AmlCompute (child class of ComputeTarget)
- 5.2 Create a Compute Target

**6.  [Image and Webservice](#ImageandWebservice)**

**7.  [Dataset](#Dataset)**

___




## 1.Workspace

### 1.1 Create a Workspace and write config.json

In [ ]:
# from azureml.core import Workspace
# ws = Workspace.create(name='myworkspace',
#             subscription_id='<azure-subscription-id>',
#             resource_group='myresourcegroup',
#             create_resource_group=True,
#             location='eastus2'
#             )
            
# ws.write_config(path="./file-path", file_name="ws_config.json")

### 1.2 Connect to Workspace with config.json

In [1]:
import azureml.core
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("This notebook was created using version 1.0.2 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")


ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

This notebook was created using version 1.0.2 of the Azure ML SDK
You are currently using version 1.0.72 of the Azure ML SDK

Workspace name: data-science
Azure region: westeurope
Subscription id: 29b64be4-867b-40ee-a259-b58b97bfc26f
Resource group: data-science


### 1.3 Connect to Workspace with variable

In [ ]:
# subscription_id = "29b64be4-867b-40ee-a259-b58b97bfc26f"
# resource_group = "data-science"
# workspace_name = "data-science"

# ws = Workspace.get(name = workspace_name,
#             subscription_id = subscription_id,
#             resource_group = resource_group)

### 1.4 Get details of the Workspace

In [3]:
ws.get_details()

{'id': '/subscriptions/29b64be4-867b-40ee-a259-b58b97bfc26f/resourceGroups/data-science/providers/Microsoft.MachineLearningServices/workspaces/data-science',
 'name': 'data-science',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': 'f3a18bbf-44bc-4e31-8355-3f8576fb0639',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-04-03T09:06:14.2884876+00:00',
 'containerRegistry': '/subscriptions/29b64be4-867b-40ee-a259-b58b97bfc26f/resourceGroups/data-science/providers/Microsoft.ContainerRegistry/registries/datascience0b4c9695',
 'keyVault': '/subscriptions/29b64be4-867b-40ee-a259-b58b97bfc26f/resourcegroups/data-science/providers/microsoft.keyvault/vaults/datascience6891748158',
 'applicationInsights': '/subscriptions/29b64be4-867b-40ee-a259-b58b97bfc26f/resourcegroups/data-science/providers/microsoft.insights/components/datascience9934152387',
 'identityPrincipalId': '8fd0b713-8c03-427a-9068-1

___
## 2. Experiment

### 2.1 Create/Get Experiment

In [2]:
experiment = Experiment(workspace=ws, name="e-commerce")
experiment

Name,Workspace,Report Page,Docs Page
e-commerce,data-science,Link to Azure Machine Learning studio,Link to Documentation


___

## 3. Run

### 3.1 Create a run object in the experiment and start logging

In [3]:
run =  experiment.start_logging()

In [4]:
# Scalar
run.log('accuracy', 0.03)
# List 
run.log_list("accuracies", [0.6, 0.7, 0.87])
# Row
run.log_row("Y over X", x=1, y=0.4)
# Table 
run.log_table("Y over X", {"x":[1, 2, 3], "y":[0.6, 0.7, 0.89]})

In [5]:
# roc curve and auc
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
# generate 2 class dataset
X, y = make_classification(n_samples=1000, n_classes=2, random_state=1)
# split into train/test sets
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2)
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(testy))]
# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(trainX, trainy)
# predict probabilities
lr_probs = model.predict_proba(testX)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(testy, ns_probs)
lr_auc = roc_auc_score(testy, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()

# Image
run.log_image('ROC', path=None, plot=plt, description='')

# show the plot
plt.show()

No Skill: ROC AUC=0.500
Logistic: ROC AUC=0.903


<Figure size 640x480 with 1 Axes>

In [6]:
# Complete the run
run.complete()

___

## 4. Model

### 4.1 Register Model

In [25]:
from sklearn import svm
import joblib
import numpy as np

# customer ages
X_train = np.array([50, 17, 35, 23, 28, 40, 31, 29, 19, 62])
X_train = X_train.reshape(-1, 1)
# churn y/n
y_train = ["yes", "no", "no", "no", "yes", "yes", "yes", "no", "no", "yes"]

clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(X_train, y_train)

joblib.dump(value=clf, filename="./model/churn-model.pkl")

['./model/churn-model.pkl']

In [26]:
from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="./model/churn-model.pkl", model_name="churn-model-test")

Registering model churn-model-test


### 4.2 Download Model

In [27]:
from azureml.core.model import Model
import os

model = Model(workspace=ws, name="churn-model-test")
model.download(target_dir=os.getcwd())

'c:\\Users\\jebaeuer\\Desktop\\repo\\azure-machine-learning-service\\churn-model.pkl'

### 4.3 Delete Model

In [ ]:
model.delete()

___

## 5. ComputeTarget, RunConfiguration, and ScriptRunConfig

### 5.1 Setting up an AmlCompute (child class of ComputeTarget)

In [30]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import AmlCompute
list_vms = AmlCompute.supported_vmsizes(workspace=ws)
# list_vms
compute_config = RunConfiguration()
compute_config.target = "amlcompute"
compute_config.amlcompute.vm_size = "STANDARD_D1_V2"

In [31]:
from azureml.core.conda_dependencies import CondaDependencies

dependencies = CondaDependencies()
dependencies.add_pip_package("scikit-learn")
dependencies.add_pip_package("numpy==1.15.4")
compute_config.environment.python.conda_dependencies = dependencies

In [34]:
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=os.getcwd(), script="./01-aml-sdk-exploration/train.py", run_config=compute_config)
experiment = Experiment(workspace=ws, name="compute_target_test")
run = experiment.submit(config=script_run_config)
run.wait_for_completion(show_output=True)

 container 1ced2732ad8d
 ---> bd3673bb828c
Step 5/15 : COPY azureml-environment-setup/99brokenproxy /etc/apt/apt.conf.d/
 ---> d903b97a5dcf
Step 6/15 : RUN if dpkg --compare-versions `conda --version | grep -oE '[^ ]+$'` lt 4.4.11; then conda install conda==4.4.11; fi
 ---> Running in d95c6fca4cfb
Removing intermediate container d95c6fca4cfb
 ---> 5fb0ae5ff9bd
Step 7/15 : COPY azureml-environment-setup/mutated_conda_dependencies.yml azureml-environment-setup/mutated_conda_dependencies.yml
 ---> 4d01357056a0
Step 8/15 : RUN ldconfig /usr/local/cuda/lib64/stubs && conda env create -p /azureml-envs/azureml_e2e47335979ab4bbc0d2a235333de638 -f azureml-environment-setup/mutated_conda_dependencies.yml && rm -rf "$HOME/.cache/pip" && conda clean -aqy && CONDA_ROOT_DIR=$(conda info --root) && rm -rf "$CONDA_ROOT_DIR/pkgs" && find "$CONDA_ROOT_DIR" -type d -name __pycache__ -exec rm -rf {} + && ldconfig
 ---> Running in be94cc4c8f7c
Solving environment: ...working... 
done


==> WARNING: A newer

### 5.2 Create a Compute Target

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


___

## 6. Image and Webservice


In [9]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script="score.py",
                                                 runtime="python",
                                                 conda_file="myenv.yml",
                                                 description="test-image-config"
                                                 )

ERROR - Execution script score.py doesn't exist. 



WebserviceException: WebserviceException:
	Message: Execution script score.py doesn't exist. 
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Execution script score.py doesn't exist. "
    }
}

In [ ]:
image = ContainerImage.create(name="test-image",
                              models=[model],
                              image_config=image_config,
                              workspace=ws
                              )

In [ ]:
from azureml.core.webservice import AciWebservice

deploy_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [ ]:
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_image(deployment_config=deploy_config,
                                       image=image,
                                       name=service_name,
                                       workspace=ws
                                       )
service.wait_for_deployment(show_output=True)

___

## 7. Dataset

### 7.1 Tabular Dataset

In [11]:
from azureml.core import Dataset

dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'customer-classification.csv')])
dataset.take(3).to_pandas_dataframe()

ValueError: String value in path must be a url starting with "http://" or "https://".

### 7.2 File Dataset

In [ ]:
from azureml.core.dataset import Dataset

url_paths = [
            'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path=url_paths)